In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/crowdflower-weather-twitter/sampleSubmission.csv
/kaggle/input/crowdflower-weather-twitter/variableNames.txt
/kaggle/input/crowdflower-weather-twitter/train.csv
/kaggle/input/crowdflower-weather-twitter/test.csv


# Import libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import shutil
import string

import tensorflow as tf
from keras import layers

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

plt.style.use("seaborn")

pd.options.display.max_rows = 2000
pd.options.display.max_columns = 500

## Load our data

In [3]:
train = pd.read_csv('/kaggle/input/crowdflower-weather-twitter/train.csv')
test = pd.read_csv('/kaggle/input/crowdflower-weather-twitter/test.csv')

## Preliminary analyse

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77946 entries, 0 to 77945
Data columns (total 28 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        77946 non-null  int64  
 1   tweet     77946 non-null  object 
 2   state     77946 non-null  object 
 3   location  66994 non-null  object 
 4   s1        77946 non-null  float64
 5   s2        77946 non-null  float64
 6   s3        77946 non-null  float64
 7   s4        77946 non-null  float64
 8   s5        77946 non-null  float64
 9   w1        77946 non-null  float64
 10  w2        77946 non-null  float64
 11  w3        77946 non-null  float64
 12  w4        77946 non-null  float64
 13  k1        77946 non-null  float64
 14  k2        77946 non-null  float64
 15  k3        77946 non-null  float64
 16  k4        77946 non-null  float64
 17  k5        77946 non-null  float64
 18  k6        77946 non-null  float64
 19  k7        77946 non-null  float64
 20  k8        77946 non-null  fl

In [5]:
train.sample(2)

,id,tweet,state,location,s1,s2,s3,s4,s5,w1,w2,w3,w4,k1,k2,k3,k4,k5,k6,k7,k8,k9,k10,k11,k12,k13,k14,k15
33627,51856,60 degrees in atl today #saywhat,georgia,Atlanta,0.0,0.0,1.000,0.000,0.000,1.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.201,0.0,0.799,0.0,0.0,0.0,0.0,0.0,0.0
60164,92697,And the party preparation began befor i even w...,iowa,"Carroll, Iowa",0.0,0.0,0.349,0.176,0.476,0.683,0.0,0.317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.348,0.0,0.652,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
for i in range(2):
    n = np.random.randint(77000)
    sample = train['tweet'][n]
    print('-'*80)
    print(f'Index: {n};\nSample:\n{sample}')

--------------------------------------------------------------------------------
Index: 69328;
Sample:
Carrie Underwood + sunshine = happiness
--------------------------------------------------------------------------------
Index: 66825;
Sample:
@mention Touche...sunny here as well....my heart's great, but it's my mouth that's recovering....silence in the neighbourhood....hehehe


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42157 entries, 0 to 42156
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        42157 non-null  int64 
 1   tweet     42157 non-null  object
 2   state     14323 non-null  object
 3   location  42144 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.3+ MB


In [8]:
test.sample(2)

,id,tweet,state,location
4737,13361,@kaseykahne dang I was telling my sisters we n...,NaN,"Middleburg, Florida"
35471,101201,why is it like 2 fucking degrees outside,delaware,"Wilmington, NC"


In [9]:
for i in range(2):
    n = np.random.randint(40000)
    sample = test['tweet'][n]
    print('-'*80)
    print(f'Index: {n};\nSample:\n{sample}')

--------------------------------------------------------------------------------
Index: 15003;
Sample:
New event. Severe Thunderstorm Warning from 5/3/2011 7:09 PM to 8:15 PM EDT for Cleveland Count... Read more at {link}
--------------------------------------------------------------------------------
Index: 30658;
Sample:
101 degrees in yoga studio today. Felt great.


## Create some function for tokenization and vectorization

In [10]:
def custom_standardization(sentence):
    sample = tf.strings.lower(sentence)
    stripped_html = tf.strings.regex_replace(sample, 'link', ' ')
    return tf.strings.regex_replace(stripped_html, 
                         '[%s]'%re.escape(string.punctuation), '')

In [11]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
                        standardize=custom_standardization,
                        max_tokens=max_features,
                        output_mode='int',
                        output_sequence_length=sequence_length)
vectorize_layer.adapt(train['tweet'])

In [12]:
for i in range(1):
    sample = np.random.randint(77000)
    print(f"Before standartization:\n{train['tweet'][sample]}")
    print('-'*80)
    print(f"After standartization:\n{custom_standardization(train['tweet'][sample])}")
    print('-'*80)
    print(f"After vectorization:\n{vectorize_layer(train['tweet'][sample])}")
    print('*'*80)

Before standartization:
"The 2liter as warm as the weather"....
--------------------------------------------------------------------------------
After standartization:
b'the 2liter as warm as the weather'
--------------------------------------------------------------------------------
After vectorization:
[ 2  1 60 70 60  2  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0

## Prepare our data to fit

In [13]:
x = train.iloc[:,1:2].values
X = np.array(vectorize_layer(x))
X.shape

(77946, 250)

In [14]:
Y = np.array(train.iloc[:,4:])
Y.shape

(77946, 24)

In [15]:
a = test.iloc[:,1:2].values
A = np.array(vectorize_layer(a))
A.shape

(42157, 250)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
print(f'X_train: {X_train.shape}, y_train: {y_train.shape}')
print('-'*50)
print(f'X_test: {X_test.shape}, y_test: {y_test.shape}')

X_train: (54562, 250), y_train: (54562, 24)
--------------------------------------------------
X_test: (23384, 250), y_test: (23384, 24)


## Build model

In [17]:
from sklearn.linear_model import Lasso

model = Lasso()

_ = model.fit(X_train,y_train)
y_pred = model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

0.2474572723329236

In [18]:
predict = model.predict(A)

## Send submission

In [19]:
subm = pd.read_csv('/kaggle/input/crowdflower-weather-twitter/sampleSubmission.csv')

In [20]:
ids = test['id']
label = pd.DataFrame(predict, columns=subm.iloc[:,1:].columns)
submission = pd.concat([ids, label], axis=1)
submission

,id,s1,s2,s3,s4,s5,w1,w2,w3,w4,k1,k2,k3,k4,k5,k6,k7,k8,k9,k10,k11,k12,k13,k14,k15
0,4,0.062468,0.272964,0.280734,0.253793,0.127889,0.771554,0.078009,0.121452,0.032627,0.020983,0.120458,0.008025,0.135479,0.050959,0.001701,0.284203,0.003436,0.076500,0.101056,0.029834,0.109298,0.162222,0.009988,0.066664
1,5,0.022896,0.103070,0.530843,0.102519,0.244039,0.657765,0.098701,0.148461,0.084850,0.022092,0.056836,0.007709,0.074521,0.081329,0.001701,0.259182,0.003436,0.088241,0.097270,0.050509,0.242230,0.096876,0.031603,0.116411
2,7,0.045376,0.167026,0.465335,0.132543,0.191098,0.716870,0.075145,0.148751,0.060216,0.023730,0.091364,0.009243,0.097394,0.114119,0.001701,0.282768,0.003436,0.091477,0.081873,0.034920,0.163237,0.107168,0.016621,0.125238
3,8,0.050135,0.205981,0.385910,0.192528,0.165057,0.742529,0.079967,0.134112,0.045647,0.021115,0.099454,0.007666,0.108777,0.089568,0.001701,0.284425,0.003436,0.085062,0.093827,0.033755,0.138743,0.135418,0.016265,0.094138
4,12,0.052789,0.230445,0.330281,0.221950,0.164474,0.723692,0.087278,0.131016,0.057771,0.026022,0.093913,0.007367,0.121986,0.041213,0.001701,0.282805,0.003436,0.075549,0.101058,0.036203,0.163953,0.148694,0.017377,0.056755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42152,120094,0.027798,0.109945,0.545939,0.116806,0.201925,0.683615,0.092864,0.138811,0.079410,0.021499,0.054992,0.008651,0.086342,0.127529,0.001701,0.259144,0.003436,0.094961,0.085945,0.044800,0.221992,0.108701,0.026481,0.131521
42153,120096,0.064078,0.294347,0.238240,0.282992,0.117448,0.777979,0.081107,0.114818,0.029528,0.022350,0.119902,0.007177,0.143579,0.025964,0.001701,0.282481,0.003436,0.069670,0.106485,0.029548,0.114119,0.175085,0.010152,0.041694
42154,120099,0.048123,0.228965,0.323368,0.224787,0.171372,0.710178,0.112073,0.119650,0.060739,0.023934,0.080641,0.007147,0.118473,0.026729,0.001701,0.243298,0.003436,0.079657,0.110635,0.035071,0.199356,0.155412,0.025309,0.051522
42155,120101,0.053375,0.236968,0.322347,0.206952,0.180025,0.726841,0.088063,0.137358,0.046292,0.021631,0.103904,0.007401,0.115693,0.033442,0.001701,0.276830,0.003436,0.072696,0.105718,0.040744,0.155466,0.136914,0.018282,0.061256


In [21]:
submission.to_csv('/kaggle/working/submission.csv', index=False)